In [3]:
import requests
import lxml.html as lh
import unicodecsv
from lxml.cssselect import CSSSelector
import requests_cache
import json

requests_cache.install_cache('demo_cache')


In [2]:
all_data = []


In [6]:
s = requests.Session()
s.mount('http://', requests.adapters.HTTPAdapter(max_retries=10))

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))
    
def stringify_children(node):
    from lxml.etree import tostring
    from itertools import chain
    parts = ([node.text] +
            list(chain(*([c.text,
                          #tostring(c,method="text",encoding="unicode"),
                          c.tail] for c in node.getchildren()))) +
            [node.tail])
    # filter removes possible Nones in texts and tails
    return ''.join(filter(None, parts))
    
def extract_data(tr):
    return [stringify_children(x).strip()
            for x in 
            tr.xpath('./*[name()="th" or name()="td"]')]


done_urls = []
    

with open('/media/sf_ddj/geschichten/2016-10-26-zeit-SpA/skilaeufer/refined/subpages.csv') as f:
    l = read_lines(unicodecsv.reader(f))
    for i,x in enumerate(l):
        if i%100==0:
            print i
        if x['url'] in done_urls:
            continue
        rq = s.get(x['url'])
        if rq.status_code!=200:
            print x['url']
            print rq
            continue
        root = lh.fromstring(rq.content)
        tables = root.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')
        if len(tables)!=1:
            print x['url'], tables
            continue
        
        tbl = tables[0]
        
        trs = tbl.xpath('.//tr')
        headers = extract_data(trs[0])
        for tr in trs[1:]:
             d = dict(zip(headers,extract_data(tr)))
             d['source_url']=x['url']
             all_data.append(d)
        done_urls.append(x['url'])
        
        if i%100=0:
            with open('done_urls.json','w') as f2:
                json.dump(done_urls, f2)

            with open('all_data.json','w') as f2:
                json.dump(all_data, f2)
                
with open('done_urls.json','w') as f2:
    json.dump(done_urls, f2)

with open('all_data.json','w') as f2:
    json.dump(all_data, f2)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


In [5]:
?json.dump

In [8]:
print len(all_data), len(done_urls)

132313 3202


In [9]:
all_keys = set()
for x in all_data:
    for k in x.keys():
        all_keys.add(k)

In [11]:
all_keys = list(all_keys)

In [ ]:
with open('fin_output.csv','w') as of:
    ow 